In [248]:
import numpy as np
import pyedflib
import statistics
import plotly.graph_objects as go
import pandas as pd
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, ForgetDimension
from gtda.plotting import plot_point_cloud, plot_heatmap, plot_diagram
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA, FastICA
from gtda.pipeline import Pipeline 

In [249]:
# Choose if you want to look at EEG or EMG data

data_type = "EEG" # Does not have an effect yet, will be added later when processing anesthesia data
#data_type = "EMG"

In [250]:
# choose individuum

subject = "m292"

In [251]:
label_list = [0, 1, 2, 3, 4]

n_folds = 5 # This should be the same as in the file which creates PDs 

In [281]:
# Load persistence diagrams

test_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values
persistence_diagrams_for_cross_validation = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values


for label in label_list:
    # Initialize dictionary with folds as keys
    train_indices_dict_for_folds[label] = {}
    validation_indices_dict_for_folds[label] = {}


for label in label_list:

    #### Final test set ####
    test_persistence_diagrams[label]  = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Test/'+str(data_type)+'/PD_Label'+str(label)+'.npy', \
    allow_pickle=True).item() # .item() to convert the dtype to dict again


    #### Data for cross validation ####
    persistence_diagrams_for_cross_validation[label] = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Train/'+str(data_type)+'/PD_Label_'+str(label)+'.npy', \
                allow_pickle=True).item() # .item() to convert the dtype to dict again


# Summary Statistics

In [446]:
def compute_summary_statistics(persistence_diagrams):
    """
    Compute summary statistics of list of persistence diagrams

    Parameters:
    - persistence_diagrams (list): persistence diagrams

    Returns:
    Tuple of four lists:
    - Persistence Entropy
    - Persistence
    - Betti Numbers
    - Complex Polynomials
    """
    
    PE = PersistenceEntropy()
    AM = Amplitude()
    NP = NumberOfPoints()
    CP = ComplexPolynomial(n_coefficients=1)

    persistence_entropies = []
    amplitudes = []
    nos_points = []
    complex_polynomials = []

    for diagram in persistence_diagrams:
        persistence_entropies.append(PE.fit_transform([diagram]))
        amplitudes.append(AM.fit_transform([diagram]))
        nos_points.append(NP.fit_transform([diagram]))
        #complex_polynomials.append(CP.fit_transform([diagram]))

    return persistence_entropies, amplitudes, nos_points, #complex_polynomials

In [447]:
def compute_largest_persistence(persistence_diagrams):
    """
    Computes persistence of the most prominent points of each dimension in each diagram

    Parameters:
    - persistence_diagrams (list): persistence diagrams

    Returns:
    List of 3 lists:
    - List of the largest persistences of homology dimension 0
    - List of the largest persistences of homology dimension 1
    - List of the largest persistences of homology dimension 2 
    """

    largest_persistences = [] # will contain 3 lists for the 3 homology dimensions
    for homology_dimension in [0, 1, 2]:
        largest_persistences_of_hom_dim = []
        for diagram in persistence_diagrams:
            # only look at holes of our homology dimension
            condition = diagram[:, 2] == homology_dimension
            filtered_diagram = diagram[condition]

            if len(filtered_diagram) > 0:
                differences = filtered_diagram[:, 1] - filtered_diagram[:, 0]
                largest_persistences_of_hom_dim.append(np.max(differences))

        largest_persistences.append(largest_persistences_of_hom_dim)

    return largest_persistences

In [448]:
# Initialize dict for final test set with labels as key
features_for_test_set = {}

# Initialize dicts for the different folds set with labels as key
features_for_cross_validation = {}

# indices
PD_indices = {}

for label in label_list:

    # Final test set features
    features_for_test_set[label] = compute_summary_statistics(test_persistence_diagrams[label][0])

    # Initialize a dict for each label for the different folds
    features_for_cross_validation[label] = []

    PD_indices[label] = persistence_diagrams_for_cross_validation[label][0].keys()

    PDs = [persistence_diagram[0] for persistence_diagram in list(persistence_diagrams_for_cross_validation[label][0].values())]

    features_for_cross_validation[label] = compute_summary_statistics(PDs)

In [449]:
# Initialize dict for final test set with labels as key
largest_persistences_for_test_set = {}

# Initialize dicts for the different folds set with labels as key
largest_persistences_for_cross_validation = {}

for label in label_list:
    
    # Final test set features
    largest_persistences_for_test_set[label] = compute_largest_persistence(test_persistence_diagrams[label][0])
    
    # Initialize a dict for each label for the different fold
    largest_persistences_for_cross_validation[label] = []

    PDs = [persistence_diagram[0] for persistence_diagram in list(persistence_diagrams_for_cross_validation[label][0].values())]
    
    largest_persistences_for_cross_validation[label] = compute_largest_persistence(PDs)

# Concatenate Features to one DataFrame

In [450]:
def choose_column_in_matrix(matrix, i):
    return [row[0][i] for row in matrix]

In [463]:
def create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, persistences, label, train, indices):
    """
    Create DataFrame for each label from features

    Parameters:
    - persistence_entropies (list): persistence entropies
    - amplitudes (list): amplitudes
    - nos_points (list): number of points
    - label (int): Label for which we want to create a dataframe. 1, 3, 5 or 7.

    Returns:
    - Feature DataFrame (DataFrame)
    """
    
    feature_df = pd.DataFrame(index= indices)

    # All 3 columns (corresponding to hole dimensions)
    feature_df[str(data_type)+"_Persistence Entropy_Dim_0"] = choose_column_in_matrix(list(persistence_entropies), 0)
    feature_df[str(data_type)+"_Persistence Entropy_Dim_1"] = choose_column_in_matrix(list(persistence_entropies), 1)
    feature_df[str(data_type)+"_Persistence Entropy_Dim_2"] = choose_column_in_matrix(list(persistence_entropies), 2)

    # All 3 columns (corresponding to hole dimensions)
    feature_df[str(data_type)+"_Amplitude_Dim_0"] = choose_column_in_matrix(list(amplitudes), 0)
    feature_df[str(data_type)+"_Amplitude_Dim_1"] = choose_column_in_matrix(list(amplitudes), 1)
    feature_df[str(data_type)+"_Amplitude_Dim_2"] = choose_column_in_matrix(list(amplitudes), 2)

    # All 3 columns (corresponding to hole dimensions)
    feature_df[str(data_type)+"_No_Points_Dim_0"] = choose_column_in_matrix(list(nos_points), 0)
    feature_df[str(data_type)+"_No_Points_Dim_1"] = choose_column_in_matrix(list(nos_points), 1)
    feature_df[str(data_type)+"_No_Points_Dim_2"] = choose_column_in_matrix(list(nos_points), 2)


    feature_df[str(data_type)+"_Largest_Persistence_Dim_0"] = persistences[0]
    feature_df[str(data_type)+"_Largest_Persistence_Dim_1"] = persistences[1]
    feature_df[str(data_type)+"_Largest_Persistence_Dim_2"] = persistences[2]

    # Label
    feature_df["Label"] = label

    # Subject
    feature_df["Subject"] = subject

    feature_df["Train"] = train


    return feature_df

In [469]:
test_dataframes= {}
dataframes_for_cross_validation = {}

for label in label_list:
    # Final test set
    persistence_entropies = features_for_test_set[label][0]
    amplitudes = features_for_test_set[label][1]
    nos_points = features_for_test_set[label][2]

    test_dataframes[label] = create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, \
                                                largest_persistences_for_test_set[label], label, False, range(len(persistence_entropies)))

    # Initialize dict with folds as key for each label
    train_dataframes[label] = {}
    validation_dataframes[label] = {}

    persistence_entropies = features_for_cross_validation[label][0]
    amplitudes = features_for_cross_validation[label][1]
    nos_points = features_for_cross_validation[label][2]
        
    
    dataframes_for_cross_validation[label] = create_feature_df(subject, data_type, persistence_entropies, amplitudes, nos_points, \
                                largest_persistences_for_cross_validation[label], label, True, PD_indices[label])

In [470]:
# Concatenate and save features of final test set
test_feature_df = pd.concat([test_dataframes[0], test_dataframes[1], test_dataframes[2], test_dataframes[3], test_dataframes[4]])
test_feature_df.to_csv("Features/"+str(subject)+"/Test/"+str(data_type)+"/Topological_Summary_Statistics.csv")


feature_df_for_cv = pd.concat([dataframes_for_cross_validation[0], dataframes_for_cross_validation[1], dataframes_for_cross_validation[2], dataframes_for_cross_validation[3], test_dataframes[4]])
feature_df_for_cv.to_csv("Features/"+str(subject)+"/Test/"+str(data_type)+"/Topological_Summary_Statistics.csv")


In [471]:
feature_df_for_cv

,EEG_Persistence Entropy_Dim_0,EEG_Persistence Entropy_Dim_1,EEG_Persistence Entropy_Dim_2,EEG_Amplitude_Dim_0,EEG_Amplitude_Dim_1,EEG_Amplitude_Dim_2,EEG_No_Points_Dim_0,EEG_No_Points_Dim_1,EEG_No_Points_Dim_2,EEG_Largest_Persistence_Dim_0,EEG_Largest_Persistence_Dim_1,EEG_Largest_Persistence_Dim_2,Label,Subject,Train
44,7.370018,6.218615,4.812837,0.084510,0.023451,0.003096,169,105,40,0.440908,0.141127,0.042094,0,m292,True
14,7.354100,6.008674,3.199372,0.090409,0.015846,0.001491,169,86,13,0.461194,0.106069,0.024146,0,m292,True
22,7.330846,6.035369,3.177352,0.084869,0.011358,0.001077,169,81,12,0.442154,0.085839,0.017536,0,m292,True
27,7.350264,6.365406,5.025669,0.133626,0.022121,0.004719,169,111,42,0.598415,0.136546,0.050558,0,m292,True
70,7.272088,4.867137,3.246767,0.147713,0.048185,0.001043,169,51,11,0.639766,0.279491,0.016340,0,m292,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,6.348380,4.600519,2.887053,0.228814,0.019690,0.000088,169,69,8,0.856504,0.108382,0.003163,4,m292,False
11,6.347517,4.361448,1.001406,0.569719,0.018577,0.002368,169,50,5,1.573436,0.116461,0.040874,4,m292,False
12,6.302491,5.069757,2.102980,0.167113,0.001465,0.000059,169,49,5,0.694622,0.024196,0.003111,4,m292,False
13,6.737135,4.005903,2.427790,0.269192,0.005404,0.000086,169,29,6,0.954516,0.063838,0.003224,4,m292,False
